# Feedback functions in NEMO Guardrails apps

TODO: This notebook is in an experimental state.

In [ ]:
# pip uninstall -y trulens_eval
# pip install git+https://github.com/truera/trulens@piotrm/azure_bugfixes#subdirectory=trulens_eval

%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

base = Path().cwd()
while not (base / "trulens_eval").exists():
    base = base.parent

print(base)

# If running from github repo, can use this:
sys.path.append(str(base))

# TODO: presently we use some features of a modified version of NeMo-Guardrails.
# Either point path to it or install from branch.
sys.path.append("/Volumes/dev_new/NeMo-Guardrails")


from trulens_eval.keys import check_keys
check_keys(
    "OPENAI_API_KEY",
    "HUGGINGFACE_API_KEY"
)

from trulens_eval import Tru
tru = Tru()
tru.reset_database()

tru.run_dashboard(_dev=base, force=True)

## Feedback function definition and registration

To make feedback functions available to rails apps, we need to first register them the `FeedbackActions` class. Note that these feedback functions do not come with selectors. The selectors are specified from within the rail app in the action execution.

In [ ]:
import logging

from pprint import PrettyPrinter
pp = PrettyPrinter()
log = logging.getLogger(__name__)

from trulens_eval import OpenAI, Huggingface, Feedback
from trulens_eval.tru_rails import TruRails, FeedbackActions
from trulens_eval.feedback.feedback import RAG_triad

# Initialize provider class
openai = OpenAI()

f_language_match = Feedback(Huggingface().language_match)

fs_triad = RAG_triad(provider=openai)

FeedbackActions.register_feedback_functions(**fs_triad)
FeedbackActions.register_feedback_functions(f_language_match)

## Rails app setup

The files created below define a configuration of a rails app adapted from
various examples in the NeMo-Guardrails repository.

In [ ]:
# Small utility to help us create various config files for rails.
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writefileinterpolated(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

Note that new additions to output rail flows in the configuration below. These are setup to run our feedback functions but their definition will come in following colang file.

In [ ]:
%%writefileinterpolated config.yaml
# Adapted from NeMo-Guardrails/nemoguardrails/examples/bots/abc/config.yml
instructions:
  - type: general
    content: |
      Below is a conversation between a user and a bot called the trulens Bot.
      The bot is designed to answer questions about the trulens_eval python library.
      The bot is knowledgeable about python.
      If the bot does not know the answer to a question, it truthfully says it does not know.

sample_conversation: |
  user "Hi there. Can you help me with some questions I have about trulens?"
    express greeting and ask for assistance
  bot express greeting and confirm and offer assistance
    "Hi there! I'm here to help answer any questions you may have about the trulens. What would you like to know?"

models:
  - type: main
    engine: openai
    model: gpt-3.5-turbo-instruct

rails:
  output:
    flows:
      - check language match
      # triad defined seperately so hopefully they can be executed in parallel
      - check rag triad groundedness
      - check rag triad relevance
      - check rag triad qs_relevance

#knowledge_base: # no longer works with openai >= 1.0
#  embedding_search_provider:
#    name: default
#    parameters:
#      embedding_engine: openai
#      embedding_model: text-embedding-ada-002

### Output flows with feedback

Next we define output flows that include checks using all 4 feedback functions we registered above.

In [ ]:
from trulens_eval.tru_rails import RailsActionSelect

# Will need to refer to these selectors/lenses to define triade checks. We can
# use these shorthands to make things a bit easier. If you are writing
# non-temporary config files, you can print these lenses to help with the
# selectors:

question_lens = RailsActionSelect.LastUserMessage
answer_lens = RailsActionSelect.BotMessage # not LastBotMessage as the flow is evaluated before LastBotMessage is available
contexts_lens = RailsActionSelect.RetrievalContexts

print(list(map(str, [question_lens, answer_lens, contexts_lens])))

In [ ]:
%%writefileinterpolated config.co
# Adapted from NeMo-Guardrails/tests/test_configs/with_kb_openai_embeddings/config.co
define user ask capabilities
  "What can you do?"
  "What can you help me with?"
  "tell me what you can do"
  "tell me about you"

define bot inform language mismatch
  "I may not be able to answer in your language."

define bot inform triad failure
  "I may may have made a mistake interpreting your question or my knowledge base."

define flow
  user ask trulens
  bot inform trulens

define parallel subflow check language match
  $result = execute feedback(\
    function="language_match",\
    selectors={{\
      "text1":"{question_lens}",\
      "text2":"{answer_lens}"\
    }},\
    verbose=True\
  )
  if $result < 0.8
    bot inform language mismatch
    stop

define parallel subflow check rag triad groundedness
  $result = execute feedback(\
    function="groundedness_measure_with_cot_reasons",\
    selectors={{\
      "statement":"{answer_lens}",\
      "source":"{contexts_lens}"\
    }},\
    verbose=True\
  )
  if $result < 0.7
    bot inform triad failure
    stop

define parallel subflow check rag triad relevance
  $result = execute feedback(\
    function="relevance",\
    selectors={{\
      "prompt":"{question_lens}",\
      "response":"{contexts_lens}"\
    }},\
    verbose=True\
  )
  if $result < 0.7
    bot inform triad failure
    stop

define parallel subflow check rag triad qs_relevance
  $result = execute feedback(\
    function="qs_relevance",\
    selectors={{\
      "question":"{question_lens}",\
      "statement":"{answer_lens}"\
    }},\
    verbose=True\
  )
  if $result < 0.7
    bot inform triad failure
    stop


## Rails app instantiation

In [ ]:
from nemoguardrails import LLMRails, RailsConfig

config = RailsConfig.from_path(".")
rails = LLMRails(config)

### Feedback action registration

We need to register the method `FeedbackActions.feedback` as an action to be able to make use of it inside the flows we defined above.

In [ ]:
rails.register_action(FeedbackActions.feedback)

## Optional `TruRails` recorder instantiation

Though not required, we can also use a trulens_eval recorder to monitor our app.

In [ ]:
tru_rails = TruRails(rails)

## Language match test invocation

Lets try to make the app respond in a different language than the question to try to get the language match flow to abort the output.

In [ ]:
# This may fail the language match:
with tru_rails as recorder:
    response = await rails.generate_async(messages=[{
        "role": "user",
        "content": "Please answer in Spanish: what does trulens_eval do?"
    }])
    
print(response['content'])

In [ ]:
# Note that the feedbacks involved in the flow are not record feedbacks hence
# not available in the usual place:

record = recorder.get()
print(record.feedback_results)

In [ ]:
# This should be ok though sometimes answers in English and the RAG triad may
# fail after language match passes.

with tru_rails as recorder:
    response = await rails.generate_async(messages=[{
        "role": "user",
        "content": "Por favor responda en español: ¿qué hace trulens_eval?"
    }])
    
print(response['content'])

## RAG triad Test

Lets check to make sure all 3 RAG feedback functions will run and hopefully
pass. Note that the "stop" in their flow definitions means that if any one of
them fails, no subsequent ones will be tested.

In [ ]:
# Should invoke retrieval:

with tru_rails as recorder:
    response = await rails.generate_async(messages=[{
        "role": "user",
        "content": "Does trulens support AzureOpenAI as a provider?"
    }])
    
print(response['content'])